In [2]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint

/Users/pablomirallesgonzalez/.micromamba/envs/master-gesbd/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
env_config = dotenv_values("../environment/.env")

In [4]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")

In [5]:
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

## Todos los resultados

In [7]:
params = {
    "index": "boe",
    "query": {"match_all": {}},
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': 'Qet6tYsBir71ATlvD31j',
           '_index': 'boe',
           '_score': 1.0,
           '_source': {'doc_id': 'BOE-A-2023-22794',
                       'text': 'Artículo\xa014.\u2003Protección de datos de '
                               'carácter personal.'}},
          {'_id': 'Qut6tYsBir71ATlvD32M',
           '_index': 'boe',
           '_score': 1.0,
           '_source': {'doc_id': 'BOE-A-2023-22794',
                       'text': '1.\u2003Los datos de carácter personal que '
                               'consten en las solicitudes y documentación '
                               'presentada serán tratados con arreglo a la Ley '
                               'Orgánica\xa03/2018, de\xa05 de diciembre, de '
                               'Protección de Datos personales y garantía de '
                               'los derechos digitales y a la disposición '
                               'adicional vigesimotercera de la Ley '
                              

## Resultados semánticos con KNN

In [15]:
query_text = "cantidad de agua tope"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': 'Oet8tYsBir71ATlvBo3Q',
           '_index': 'boe',
           '_score': 0.7803806,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': 'Artículo\xa069.\u2003 Requisitos específicos '
                               'de liquidez.'}},
          {'_id': '7ut8tYsBir71ATlvY49x',
           '_index': 'boe',
           '_score': 0.7126442,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': 'c)\u2003el volumen de la operación,'}},
          {'_id': '4ut8tYsBir71ATlvYY_3',
           '_index': 'boe',
           '_score': 0.7126442,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': 'c)\u2003el volumen de la operación,'}},
          {'_id': 'Put8tYsBir71ATlvB42z',
           '_index': 'boe',
           '_score': 0.7086704,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': '3.\u2003La CNMV determinará el nivel de '
                               're

## Combinado

In [14]:
query_text = "cantidad de agua"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
    "filter": {
        "match": {
            "text": "liquidez",
        },
    },
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': '9-t7tYsBir71ATlvmIlH',
           '_index': 'boe',
           '_score': 0.7720374,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': 'Artículo\xa069.\u2003Requisitos específicos de '
                               'liquidez.'}},
          {'_id': 'Oet8tYsBir71ATlvBo3Q',
           '_index': 'boe',
           '_score': 0.7720374,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': 'Artículo\xa069.\u2003 Requisitos específicos '
                               'de liquidez.'}},
          {'_id': 'Put8tYsBir71ATlvB42z',
           '_index': 'boe',
           '_score': 0.70936024,
           '_source': {'doc_id': 'BOE-A-2023-22763',
                       'text': '3.\u2003La CNMV determinará el nivel de '
                               'requisitos específicos de liquidez, como la '
                               'diferencia entre la liquidez considerada '
                               'suficiente con arreg